In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as reg
import pickle
import pandas as pd

In [ ]:
https://www.alibaba.com/trade/search?spm=a2700.supplier-normal.16.4.3221534b7b4eyb&page=1&indexArea=company_en&n=38&keyword=women_dresses&f1=y&viewType=L&ta=y

In [ ]:
start = time.time()

main_url0 = 'https://www.alibaba.com/trade/search?spm=a2700.supplier-normal.16.4.3221534b7b4eyb&page='
main_url1 = '&indexArea=company_en&n=38&keyword=women_dresses&f1=y&viewType=L&ta=y'
DataSet = []
j = 1
while(j<=52):
    
    uClient = reg(main_url0+str(j)+main_url1)
    html_page = uClient.read()
    uClient.close()
    
    page_soup = soup(html_page,"lxml")
    containers = page_soup.findAll("div" , {"class","company"})
    companies = len(containers)
    
    k = 0
    while(k<companies):
        Data = {}
        Data['Url'] = containers[k].find('a',{'class','cd'}).get('href')
        DataSet.append(Data)

        k+=1
    j+=1
end = time.time()
print(end - start)

In [ ]:
len(DataSet)

In [ ]:
df = pd.DataFrame(DataSet)
df.to_csv('Companies Contact URL.csv',index=False)

In [2]:
Contacts = pd.read_csv('Companies Contact URL.csv')
Contacts.head()

,Url
0,http://dgyihao.en.alibaba.com/contactinfo.html
1,http://candowclothing.en.alibaba.com/contactin...
2,http://lover-beauty.en.alibaba.com/contactinfo...
3,http://nixiya.en.alibaba.com/contactinfo.html
4,http://liule.en.alibaba.com/contactinfo.html


In [3]:
driver = webdriver.Chrome(r'C:\Users\Arslan Farooq\Documents\ChromeDriver\chromedriver.exe')
driver.get("http://dgyihao.en.alibaba.com/contactinfo.html")
driver.maximize_window()

In [ ]:
cookies = driver.get_cookies()
pickle.dump( cookies , open("cookies.pkl","wb"))

In [8]:
def Loading(url):
    driver.get(url)
    driver.maximize_window()
    return driver
Companies_Data = []

In [ ]:
https://xiante.en.alibaba.com/contactinfo.html
https://kimiko.en.alibaba.com/contactinfo.html
https://miaozixin.en.alibaba.com/contactinfo.html

In [11]:
number_of_links = len(Contacts)
s = time.time()
i = 1960
while(i<number_of_links):
    Data = {}
    driver = Loading(Contacts['Url'][i])
    
    #Upload Cookies
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    
    #Find View Details Button if view details button then goto try part otherwise into except part
    try:
        view_details = driver.find_element_by_link_text("View details")
        view_details.click()
        
        # Convert page into soup
        page_soup = soup(driver.page_source,'lxml')
        
        # Find the info-table 
        try:
            info_table = page_soup.find('table',{'class','info-table'})
            trs = info_table.find_all('tr')
            
            # check if the trs are equal to or greater than 7
            if(len(trs)>=7):
                try:
                    Data['Mobile Number'] = trs[1].find('td').text
                except:
                    Data['Mobile Number'] = None
                #end try-except
            else:
                Data['Mobile Number'] = None
            #end if-else
            try:
                Data['Name'] =  page_soup.find('div',{'class','contact-name'}).text
            except:
                Data['Name'] = None
                #end try-except
            try:
                Data['Title'] =  page_soup.find('div',{'class','contact-job'}).text
            except:
                Data['Title'] = None
                #end try-except   
            try:
                contact_table = page_soup.find('table',{'class','contact-table'})
                trss = contact_table.find_all('tr') 
                
                if(len(trss)>3):
                    try:
                        Data['Company Name'] =  trss[0].find('td').text
                    except:
                        Data['Company Name'] =None
                    try:
                        Data['Gernal Websit'] =  trss[2].find('td').text
                    except:
                        Data['Gernal Websit'] =None
                    try:
                        Data['Alibaba Website'] = trss[3].find('td').text
                    except:
                        Data['Alibaba Website'] =None
                else:
                    try:
                        Data['Company Name'] =  trss[0].find('td').text
                    except:
                        Data['Company Name'] =None
                    try:
                        Data['Alibaba Website'] = trss[2].find('td').text
                        Data['Gernal Websit'] =None
                    except:
                        Data['Alibaba Website'] =None
            except:
                Data['Company Name'] =None
                Data['Gernal Websit'] =None
                Data['Alibaba Website'] =None
                print("Company Contact Information Not Found")
            
        except:
            
            try:
                div = page_soup.find('div',{'class','contact-detail'})
                dd = div.find_all('dd')
                Data["Mobile Number"] = dd[1].text
            except:
                Data['Mobile Number'] = None
            try:
                name = page_soup.find('h1',{'class','name'}).text
                Data['Name'] = name.replace('\t','').replace('\n','')
            except:
                Data['Name'] = None
            try:
                Info = page_soup.find('dl',{'class','dl-horizontal'}).text
                title = Info.replace('\n', '')
                t = title.split(':')
                Data['Title'] =  t[2]
            except:
                Data['Title'] = None
            
            try:
                contact_table = page_soup.find('table',{'class','company-info-data table'})
                trss = contact_table.find('tbody')  
                
                if(len(trss)>3):
                    try:
                        Data['Company Name'] =  trss.find_all('td')[1].text
                    except:
                        Data['Company Name'] =None
                    try:
                        Data['Gernal Websit'] =  trss.find_all('td')[5].text.replace('\t','').replace('\n','')
                    except:
                        Data['Gernal Websit'] =None
                    try:
                        Data['Alibaba Website'] = trss.find_all('td')[7].text
                    except:
                        Data['Alibaba Website'] =None
                else:
                    try:
                        Data['Company Name'] =  trss.find_all('td')[1].text
                    except:
                        Data['Company Name'] =None
                    try:
                        Data['Alibaba Website'] = trss.find_all('td')[7].text
                        Data['Gernal Websit'] =None
                    except:
                        Data['Alibaba Website'] =None
            except:
                Data['Company Name'] =None
                Data['Gernal Websit'] =None
                Data['Alibaba Website'] =None
                print(" Not Found Company Contact Information")                
    except:
                
        # Convert page into soup
        page_soup = soup(driver.page_source,'lxml')
        Data['Mobile Number'] = None

        try:
            Info = page_soup.find('dl',{'class','dl-horizontal'}).text
            title = Info.replace('\n', '')
            t = title.split(':')
            Data['Title'] =  t[2]
        except:
            Data['Title'] = None
            
        try:
            contact_table = page_soup.find('table',{'class','contact-table'})
            trss = contact_table.find_all('tr') 
            if(len(trss)>3):
                try:
                    Data['Company Name'] =  trss[0].find('td').text
                except:
                    Data['Company Name'] =None
                try:
                    Data['Gernal Websit'] =  trss[2].find('td').text
                except:
                    Data['Gernal Websit'] =None
                try:
                    Data['Alibaba Website'] = trss[3].find('td').text
                except:
                    Data['Alibaba Website'] =None
            else:
                try:
                    Data['Company Name'] =  trss[0].find('td').text
                except:
                    Data['Company Name'] =None
                try:
                    Data['Alibaba Website'] = trss[2].find('td').text
                    Data['Gernal Websit'] =None
                except:
                    Data['Alibaba Website'] =None
        except:
            Data['Company Name'] =None
            Data['Gernal Websit'] =None
            Data['Alibaba Website'] =None
            print("Company Contact Information Not Found without view details")
    
    Companies_Data.append(Data)
    print('Index', i)
    i+=1

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=53320): Max retries exceeded with url: /session/92361b5d746427429a84838870df65a2/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D653A92A20>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [10]:
len(Companies_Data)

1960

In [12]:
df = pd.DataFrame(Companies_Data)
df.head(20)

,Alibaba Website,Company Name,Gernal Websit,Mobile Number,Name,Title
0,dgyihao.en.alibaba.com,"Dongguan Humen Yihao Clothing Co., Ltd.",http://www.yihaofashion.cn,86-18699992688,Mr. Liuwu Peng,sales manager
1,candowclothing.en.alibaba.com,"Suzhou Chuan Zai Gu Clothing Co., Ltd.",http://www.retrodesign.cn,None,Ms. Gillian Lu,None
2,lover-beauty.en.alibaba.com,"Quanzhou Hexin Fashion Co., Ltd.",http://www.lover-beauty.com,86-18559680861,Ms. Alice Gong,Sales Manager
3,nixiya.en.alibaba.com,"Guangzhou Nixiya Garment Co., Ltd.",http://www.nixiyagarment.com,15768883077,Ms. Rita Liao,Sales
4,liule.en.alibaba.com,"Guangzhou Liule Garments Co., Ltd.",http://www.yileya.com.cn,8618588711219,Mr. Jay Chen,Sales
5,hxlingerie.en.alibaba.com,"Yiwu City Hexin Lingerie Co., Ltd.",http://www.feelingirldress.com,18559680845,Ms. Kelly Cai,CEO
6,iofferlingerie.en.alibaba.com,"Yiwu City Hexin Lingerie Co., Ltd.",http://www.dreamingirl.com/,008615249094965,Ms. Emily Ji,Sales Manager
7,winnerclothing.en.alibaba.com,"Shenzhen Winner Clothing Co., Ltd.",http://www.winnerclothing.cn,None,Mr. Winner Clothing SZ,sales manager
8,kaiyalaceromper.en.alibaba.com,"Yiwu Kaiya E-Commerence Co., Ltd.",http://www.kaiyawholesale.comhttp://www.aliexp...,8613626696955,Ms. Lily Hou,Sales woman
9,hongyuhy.en.alibaba.com,"Dongguan City Hongyu Apparel Co., Ltd.",https://hongyuhy.en.alibaba.com,18825803155,Ms. Lydia Meng,salesman


In [13]:
df.isnull().sum()

Alibaba Website      2
Company Name         2
Gernal Websit      498
Mobile Number      749
Name               246
Title              856
dtype: int64

In [ ]:
len(df['Mobile Number'][13])

In [14]:
j=0
while(j<len(df['Mobile Number'])):
    if(df['Mobile Number'][j]!=None):
        l = len(df['Mobile Number'][j]) 
        if(l==14):
            df['Mobile Number'][j] = df['Mobile Number'][j][3:]
        elif(l==13):
            df['Mobile Number'][j] = df['Mobile Number'][j][2:]
        elif(l==15):
            df['Mobile Number'][j] = df['Mobile Number'][j][4:]
        elif(l==16):
            df['Mobile Number'][j] = df['Mobile Number'][j][4:]
    j+=1
df.head(20)

,Alibaba Website,Company Name,Gernal Websit,Mobile Number,Name,Title
0,dgyihao.en.alibaba.com,"Dongguan Humen Yihao Clothing Co., Ltd.",http://www.yihaofashion.cn,18699992688,Mr. Liuwu Peng,sales manager
1,candowclothing.en.alibaba.com,"Suzhou Chuan Zai Gu Clothing Co., Ltd.",http://www.retrodesign.cn,None,Ms. Gillian Lu,None
2,lover-beauty.en.alibaba.com,"Quanzhou Hexin Fashion Co., Ltd.",http://www.lover-beauty.com,18559680861,Ms. Alice Gong,Sales Manager
3,nixiya.en.alibaba.com,"Guangzhou Nixiya Garment Co., Ltd.",http://www.nixiyagarment.com,15768883077,Ms. Rita Liao,Sales
4,liule.en.alibaba.com,"Guangzhou Liule Garments Co., Ltd.",http://www.yileya.com.cn,18588711219,Mr. Jay Chen,Sales
5,hxlingerie.en.alibaba.com,"Yiwu City Hexin Lingerie Co., Ltd.",http://www.feelingirldress.com,18559680845,Ms. Kelly Cai,CEO
6,iofferlingerie.en.alibaba.com,"Yiwu City Hexin Lingerie Co., Ltd.",http://www.dreamingirl.com/,15249094965,Ms. Emily Ji,Sales Manager
7,winnerclothing.en.alibaba.com,"Shenzhen Winner Clothing Co., Ltd.",http://www.winnerclothing.cn,None,Mr. Winner Clothing SZ,sales manager
8,kaiyalaceromper.en.alibaba.com,"Yiwu Kaiya E-Commerence Co., Ltd.",http://www.kaiyawholesale.comhttp://www.aliexp...,13626696955,Ms. Lily Hou,Sales woman
9,hongyuhy.en.alibaba.com,"Dongguan City Hongyu Apparel Co., Ltd.",https://hongyuhy.en.alibaba.com,18825803155,Ms. Lydia Meng,salesman


In [ ]:
j=0
while(j<len(df['Mobile Number'])):
    if(df['Mobile Number'][j]!=None):
        l = len(df['Mobile Number'][j]) 
        if(l==14):
            df['Mobile Number'][j] = df['Mobile Number'][j][3:]
        elif(l==13):
            df['Mobile Number'][j] = df['Mobile Number'][j][2:]
    j+=1
df.head()

In [ ]:
733

Index 540
Company Contact Information Not Found without view details
Index 541

Index 1800
Company Contact Information Not Found without view details
Index 1801

847

1252

In [16]:
df.to_csv("Companies_Info.csv", index=False)

In [ ]:
print(len(df['Mobile Number'][0]))
print(df['Mobile Number'][0])

print(len(df['Mobile Number'][4]))
print(df['Mobile Number'][4])


In [ ]:
if(df['Mobile Number'][1]!=None):
    a = len(df['Mobile Number'][1])
    a
else:
    print('good')

In [ ]:
j=0
while(j<len(df['Mobile Number'])):
    if(df['Mobile Number'][j]!=None):
        l = len(df['Mobile Number'][j]) 
        if(l==14):
            df['Mobile Number'][j] = df['Mobile Number'][j][3:]
        elif(l==13):
            df['Mobile Number'][j] = df['Mobile Number'][j][2:]
    j+=1

In [ ]:
df

In [ ]:
df['Mobile Number'][2]

In [ ]:
df['Mobile Number'][2][3:]

In [ ]:
df['Mobile Number'][10]

In [ ]:
len(df['Mobile Number'][10])

In [ ]:
df['Mobile Number'][10][2:]